In [36]:
!pip -qqq install pip --progress-bar off

!pip install -q torch datasets --progress-bar off
!pip install -q accelerate peft bitsandbytes trl --progress-bar off

!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off
!pip install groq --progress-bar off
!pip install rank_bm25


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastembed 0.2.7 requires huggingface-hub<0.21,>=0.20, but you have huggingface-hub 0.24.5 which is incompatible.
transformers 4.42.4 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastembed 0.2.7 requires huggingface-hub<0.21,>=0.20, but you have huggingface-hub 0.24.5 which is incompatible.
fastembed 0.2.7 requires tokenizers<0.16,>=0.15, but you have tokenizers 0.19.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.20.0 requires huggingface-hu

In [37]:
!pip -qqq install langchain-groq==0.1.3 --progress-bar off

In [38]:

import os
import textwrap
from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse



from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
os.environ["GROQ_API_KEY"] = 

# def print_response(response):

def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))


In [89]:
# Updated Document Parsing Instruction
instruction = """The provided documents collectively offer a comprehensive history of Nepal, spanning key periods, events, dynasties, and cultural developments. These documents include detailed accounts of various ruling periods, such as the Malla era, and the evolution of Nepal's infrastructure, society, and governance.

When answering questions based on these documents, please follow these guidelines:
- Be precise and concise in your responses.
- Ensure that the information is accurate and directly relevant to the question.
- When referencing specific periods or events, highlight relevant sections or passages where applicable.
- Provide context to your answers when necessary, explaining the implications or importance of specific events or developments.
- Maintain a neutral and informative tone, avoiding any personal opinions or interpretations.

The goal is to provide clear and informative answers that help the user understand various aspects of Nepal's history."""

parser = LlamaParse(
    api_key = "",
    result_type = 'markdown',
    parsing_instruction = instruction,
    max_timeout = 5000,

)
documents = [
    '/content/drive/My Drive/HistoryofNepal_Daniel.pdf',
    '/content/drive/My Drive/HistoryofNepal.pdf',
    '/content/drive/My Drive/MallaKings.pdf',
    '/content/drive/My Drive/AccountofNepal.pdf'
]
for document_path in documents:
    llama_parse_document = await parser.aload_data(document_path)


Started parsing the file under job_id 485221e2-3b30-4f15-818d-167349bd4cb0
Started parsing the file under job_id 8ae4fca4-f152-4277-ac9e-c11ad06c0357
Started parsing the file under job_id d093514c-00c4-4dc1-a0ec-fbb23530155c
Started parsing the file under job_id 4e93bd1c-c8de-40e1-8cfe-a22ae98542e5


In [91]:

parsed_doc = llama_parse_document[0]
# Markdown(parsed_doc.text[1000:])

In [92]:

from pathlib import Path

document_path = Path('/content/drive/MyDrive/all_doc_parsed_document.md')
with document_path.open('a') as f:
    f.write(parsed_doc.text)



### Vector embeddings

In [93]:
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

In [94]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2048, chunk_overlap = 128)
documents = text_splitter.split_documents(loaded_documents)
print(documents[2].page_content)

At Home with Apartheid

Ginsburg, who teaches landscape architecture at the University of Illinois, Urbana-Champaign, also takes note of the social divisions among white South Africans. Far from appearing monolithic in their beliefs, they are shown to have exercised different attitudes toward their domestic servants based on their own experiences, which were rooted in differing ethics formed by class, ethnicity, language, religion, and gender. The homes and servant quarters described by the author manifest the different aspirations and anxieties of the women and men who inhabited them, reflecting the broader social conditions of a nation divided by race, by gender, and by class.

The difficulty of bridging two worlds—one rooted in the village life of largely Bantu-speaking people, the other a cosmopolitan realm of global commerce and culture—introduces and concludes At Home with Apartheid. Ginsburg notes the shock experienced by women who migrated from villages (or the black townships 

In [96]:
embeddings = FastEmbedEmbeddings(model_name = "BAAI/bge-base-en-v1.5")
qdrant = Qdrant.from_documents(documents,
                               embeddings,
                               path = '/content/drive/MyDrive/qdrant_db',
                               collection_name = 'History0fNepal_embeddings'
                               )

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

# perform dense retrieval using qdrant

In [97]:
%%time

query = "Who were Malla?"
similar_docs = qdrant.similarity_search_with_score(query)
dense_result = [(doc.page_content, score) for doc, score in similar_docs]

CPU times: user 698 ms, sys: 43.6 ms, total: 742 ms
Wall time: 160 ms


In [98]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("_" * 80)
    print()


text: This dynasty, whatever may have been its origin, was expelled by the Kerrats (Kiratas), of whom 27 princes governed 1630 years. The names of these princes, as might be expected from what I have said of the tribe Kirata, are entirely barbarous.

The barbari

score: 0.6059401730081675
________________________________________________________________________________

text: Vol.I.p.311.

ACCOUNTOFNEPAL. Chap. r.

enormous reigns of Narasingha and Ramsingha. At any rate, if the people of Garsamaran retired to Nepal, and became the Newars, the 1322 (or 1323, as Colonel Kirkpatrick lias it,) * is the most probable date of the ev

score: 0.6019400639028967
________________________________________________________________________________

text: The Bhotiyas would again govern 490 years, beginning A.D. 833.

In the account of the Newars I have already stated, that this people totally deny their having come from the plains, or that their princes were descended of Hari Singhailt of Gar Samaran

In [99]:
%%time
retriever = qdrant.as_retriever(search_kwargs = {"k": 5})
retrieved_docs = retriever.invoke(query)


CPU times: user 725 ms, sys: 19.9 ms, total: 745 ms
Wall time: 161 ms


In [100]:

for doc in retrieved_docs:
  print(f"id: {doc.metadata['_id']}\n")
  print(f"text: {doc.page_content[:256]}")
  print("_" * 80)
  print()


id: 9d3c4e597874475781c739bde6455346

text: This dynasty, whatever may have been its origin, was expelled by the Kerrats (Kiratas), of whom 27 princes governed 1630 years. The names of these princes, as might be expected from what I have said of the tribe Kirata, are entirely barbarous.

The barbari
________________________________________________________________________________

id: 5d05b353c7d047389effff8181ea3ce3

text: Vol.I.p.311.

ACCOUNTOFNEPAL. Chap. r.

enormous reigns of Narasingha and Ramsingha. At any rate, if the people of Garsamaran retired to Nepal, and became the Newars, the 1322 (or 1323, as Colonel Kirkpatrick lias it,) * is the most probable date of the ev
________________________________________________________________________________

id: 5bcb8c6582ba4e1b86156c75e2b706c9

text: The Bhotiyas would again govern 490 years, beginning A.D. 833.

In the account of the Newars I have already stated, that this people totally deny their having come from the plains, or that t

# perform sparse retrieval using BM25

In [101]:
from rank_bm25 import BM25Okapi

texts = [doc.page_content for doc in documents]
tokenized_text = [text.split() for text in texts]
bm25 = BM25Okapi(tokenized_text)



In [102]:
def bm25_search(query, bm25, texts):
    tokenized_query = query.split(" ")  # Tokenize the query
    scores = bm25.get_scores(tokenized_query) # Get BM25 scores for each document
    results = sorted(zip(scores, texts), reverse=True) # Combine scores and texts, sort by scores
    return results

In [103]:
# Execute BM25 search
bm25_results = bm25_search(query, bm25, texts)

In [104]:
# Combine dense and BM25 results, ensuring scores are float
combined_results = [(text, float(score)) for text, score in dense_result] + [(text, float(score)) for score, text in bm25_results]

# Sort combined results by score (in descending order)
combined_results.sort(key=lambda x: x[1], reverse=True)

# Print the combined results
for text, score in combined_results:
    print(f"text: {text[:256]}\n")
    print(f"score: {score}")
    print("_" * 80)
    print()


text: This dynasty, whatever may have been its origin, was expelled by the Kerrats (Kiratas), of whom 27 princes governed 1630 years. The names of these princes, as might be expected from what I have said of the tribe Kirata, are entirely barbarous.

The barbari

score: 0.6059401730081675
________________________________________________________________________________

text: Vol.I.p.311.

ACCOUNTOFNEPAL. Chap. r.

enormous reigns of Narasingha and Ramsingha. At any rate, if the people of Garsamaran retired to Nepal, and became the Newars, the 1322 (or 1323, as Colonel Kirkpatrick lias it,) * is the most probable date of the ev

score: 0.6019400639028967
________________________________________________________________________________

text: The Bhotiyas would again govern 490 years, beginning A.D. 833.

In the account of the Newars I have already stated, that this people totally deny their having come from the plains, or that their princes were descended of Hari Singhailt of Gar Samaran

In [106]:
# Rerank and compress
compressor = FlashrankRerank(model = 'ms-marco-MiniLM-L-12-v2')
compression_retriever = ContextualCompressionRetriever(
    base_compressor = compressor,
    base_retriever = retriever)

In [107]:
%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

CPU times: user 41 µs, sys: 0 ns, total: 41 µs
Wall time: 83 µs
Running pairwise ranking..


3

In [108]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()


id: c736eb0fb3c649b59b56e8cc918fec14

text: Nepaul, pp. 243-252.
Nepaul, p. 123.

Chap-L ACCOUNT OF NEPAL. 27

eludesthemamongtheamonetha Ksbatriya4{,h ormilitary cast Buthitherto the tribehasbeensopowerful,that manypeopleinthe westspeak itslanguage althoughthey do not belong to it; and by farthe gr

score: 0.0015113061526790261
--------------------------------------------------------------------------------

id: 082b4e75b3b2452faa9885948a7d9cca

text: 500 to 1500 rupees, according as the cultivators werediscontented or satisfied. Some of the officers had lands free of rent on the plains; but, as usual among the mountaineers, the whole revenue from the plain went to the chief. On the hills of this state 

score: 0.0009041653829626739
--------------------------------------------------------------------------------

id: 9d3c4e597874475781c739bde6455346

text: This dynasty, whatever may have been its origin, was expelled by the Kerrats (Kiratas), of whom 27 princes governed 1630 years. Th

In [109]:
llm = ChatGroq(temperature = 0, model = 'mixtral-8x7b-32768')


In [110]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""


In [111]:
prompt = PromptTemplate(template = prompt_template,
                        input_variables = ['context' , 'question'] )


In [112]:
qa = RetrievalQA.from_chain_type(llm = llm,
                                chain_type = 'stuff',
                                retriever = compression_retriever,
                                chain_type_kwargs = {'prompt': prompt, "verbose": False})

In [113]:
%%time
response = qa.invoke("Where is Nepal?")


Running pairwise ranking..
CPU times: user 18.6 s, sys: 5.38 s, total: 23.9 s
Wall time: 1.28 s


In [114]:

print_response(response)


Nepal is a territory in the Himalayas, occupying about thirteen degrees of longitude and five
degrees of latitude. It is named after its capital, Kathmandu, but is commonly used to refer to the
whole territory of the Gorkha Rajas. To the east of Nepal Proper, there is a tribe called Kirat
Kichak, who were once a powerful nation but were subject to Rajput princes by 1769. The Kirats were
the principal strength of these Rajput chiefs, and their hereditary chief held the second office in
the state.

Additional information:
Nepal is home to Parbat or Malebum, a very elevated and cold country with mines of sulphur,
cinnabar, iron, copper, and some say zinc. It also has hot springs at Muktanath and rock crystals.
Gold is found in the sands of several rivers, especially the Krishna Gandaki or Narayani, the
Bakhugar or Bathugar, the Modi, and the Mayangdi. The upper part of the river, called the Kali, runs
through the territories of a Bhotiya chief called the Mastang Raja, who was tributary to

In [115]:
%%time
response = qa.invoke("Where did the name of Nepal came from?")

Running pairwise ranking..
CPU times: user 18.4 s, sys: 5.86 s, total: 24.2 s
Wall time: 991 ms


In [116]:

print_response(response)

The name of Nepal is derived from a certain Niyam, a Muni, or very holy person, according to a
modern conceit. However, both south and north Brahmans agree in writing the name as Nepal. Some
people who inhabit near Kathmandu call themselves Sayn, and the same name is given by the Newars to
the whole nation. Thibet is a Persian word, and the native appellation of the territory subject to
Lassa is Borka, according to a patient the responder had who had been chief of a territory north
from Lassa. The Lamas are the priests of the sect of Bouddh in Thibet and the adjacent territories,
and they reject the doctrine of caste.


In [117]:
%%time
response = qa.invoke("How is prithvi Narayan Shah?")

Running pairwise ranking..
CPU times: user 18.6 s, sys: 5.74 s, total: 24.4 s
Wall time: 1.26 s


In [118]:

print_response(response)

Based on the information provided, Prithwi Narayan Shah is no longer alive. He died around 1771. He
was the king of Gorkha and is known for conquering Nepal and unifying its smaller principalities. He
was succeeded by his son, Singha Pratap. Prithwi Narayan Shah is also known for introducing
firelocks and European discipline to his army. However, his jealousy of Europeans prevented him from
employing them in his service. This decision might have hindered the progress of his troops in
tactics and the dexterous use of their arms.

Additional information: Prithwi Narayan Shah had two legitimate sons, Singha Pratap and another son
whose name is not mentioned in the text. He also had a beautiful daughter who rejected a suitor due
to his low rank but later married Prithwi Narayan Shah. The king was able to conquer Nepal due to
his personal endowments and the use of firelocks. However, his relationship with the European
character was strained, and he advised his successors not to employ Europ

In [119]:
%%time
response = qa.invoke("Who are Mallas?")

Running pairwise ranking..
CPU times: user 16.6 s, sys: 5.71 s, total: 22.3 s
Wall time: 1.18 s


In [120]:

print_response(response)

The Mallas were a dynasty that governed Nepal for 175 years, according to the given information.
They were descendants of Niyam Muni and had Sanskrit names with the addition of Gupt to mark their
descent and tribe. The Malla dynasty was preceded by the rule of the Kiratas and followed by the
rule of the Newars. Later, the Bhotiyas would again govern Nepal, beginning in A.D. 833. The
information provided does not indicate any further details about the Mallas.

Additional information:

* The Kiratas were a barbarian tribe who expelled the previous rulers of Nepal and governed for 1630
years.
* The Newars were originally the sovereigns of Nepal and later recovered their dominions.
* The Bhotiyas were a tribe who governed Nepal for 490 years, beginning in A.D. 833.
* The Mallas were a dynasty that governed Nepal for 175 years, and they were descendants of Niyam
Muni.
* The Newar dynasty was established by a great conqueror named Jat Mull in the sixth generation from
the founder of the Newa

In [123]:
%%time
response = qa.invoke("Who was Jung Bahadur Rana?")


Running pairwise ranking..
CPU times: user 19.1 s, sys: 5.79 s, total: 24.8 s
Wall time: 963 ms


In [124]:

print_response(response)

Based on the provided text, there is no direct mention of a person named Jung Bahadur Rana. It is
possible that he is not a key figure in the account of Nepal being discussed. However, the text does
provide information about the rulers and political situation in Nepal during the late 18th century.
This includes details about Rana Bahadur, who was a ruler during this time. It is important to note
that the text does not provide enough information to accurately answer the question about Jung
Bahadur Rana.


In [125]:
%%time
response = qa.invoke("Tell me something on King Birendra Bir Bikram Shah Dev?")


Running pairwise ranking..
CPU times: user 20.6 s, sys: 5.42 s, total: 26 s
Wall time: 996 ms


In [126]:

print_response(response)

I'm sorry for any confusion, but the information provided does not include details about King
Birendra Bir Bikram Shah Dev. He was the King of Nepal from 1972 to 2001. His reign was marked by a
period of political stability and economic growth. However, the information given here focuses on
the history of the principality founded by Lohangga Sen and the roles of different leaders in its
government. I hope this information is still helpful to you in some way. If you have any other
questions, please let me know.


In [127]:
%%time
response = qa.invoke("Tell me something on Shah Kings")


Running pairwise ranking..
CPU times: user 20.1 s, sys: 5.43 s, total: 25.5 s
Wall time: 1.12 s


In [128]:

print_response(response)

The Shah Kings are not directly mentioned in the provided text. However, it is known that the Shah
dynasty came to power in Nepal in 1768, after the overthrow of the Malla Kings. The first Shah King
was Prithvi Narayan Shah. The text does mention an attack by Prithwi Narayan Shah on Kama Sen, a
prince from a different dynasty, in 1773. This could be an early event in the Shah Kings' campaign
to unify and expand their kingdom.

Additional information:
The Shah dynasty ruled Nepal until 2008, when the country became a federal democratic republic. The
Shah Kings are known for their military conquests and the unification of Nepal into a single state.
Prithvi Narayan Shah is considered the father of modern Nepal. The Shah Kings also promoted Hinduism
as the state religion and introduced the Muluki Ain, a legal code, in 1854.
